In [1]:
import pandas as pd 
import numpy as np
import os 
pd.options.display.max_colwidth = 10000
artists = "artists.dat"
tags = "tags.txt"
user_artists = "user_artists.dat"
user_friends = "user_friends.dat"
user_taggedartists_timestamps = "user_taggedartists-timestamps.dat"
user_taggedartists = "user_taggedartists.dat"

**Data statistics**
---------------
   1892 users
   17632 artists
      
   12717 bi-directional user friend relations, i.e. 25434 (user_i, user_j) pairs
         avg. 13.443 friend relations per user
         
   92834 user-listened artist relations, i.e. tuples [user, artist, listeningCount]
         avg. 49.067 artists most listened by each user
         avg. 5.265 users who listened each artist
            
   11946 tags  
   
  186479 tag assignments (tas), i.e. tuples [user, tag, artist]
         avg. 98.562 tas per user
         avg. 14.891 tas per artist
         avg. 18.930 distinct tags used by each user
         avg. 8.764 distinct tags used for each artist

**Files**
----- 
   * artists.dat
   
    This file contains information about music artists listened and tagged by the users.
   
   * tags.dat
   
   	This file contains the set of tags available in the dataset.

   * user_artists.dat
   
        This file contains the artists listened by each user.
        
        It also provides a listening count for each [user, artist] pair.

   * user_taggedartists.dat - user_taggedartists-timestamps.dat
   
        These files contain the tag assignments of artists provided by each particular user.
        
        They also contain the timestamps when the tag assignments were done.
   
   * user_friends.dat
   
   	These files contain the friend relations between users in the database.
     

In [2]:
print(pd.read_table(user_artists).head())
print("---------------------------------")
print(pd.read_table(user_taggedartists).head())
print("---------------------------------")
print(pd.read_table(tags,encoding = "utf-8").head())
print("---------------------------------")
print(pd.read_table(user_friends).head())
print("---------------------------------")
print(pd.read_table(artists).head())
print("---------------------------------")
#print(pd.read_table(user_taggedartists_timestamps).head()) #don't need timestamps

   userID  artistID  weight
0       2        51   13883
1       2        52   11690
2       2        53   11351
3       2        54   10300
4       2        55    8983
---------------------------------
   userID  artistID  tagID  day  month  year
0       2        52     13    1      4  2009
1       2        52     15    1      4  2009
2       2        52     18    1      4  2009
3       2        52     21    1      4  2009
4       2        52     41    1      4  2009
---------------------------------
   tagID           tagValue
0      1              metal
1      2  alternative metal
2      3          goth rock
3      4        black metal
4      5        death metal
---------------------------------
   userID  friendID
0       2       275
1       2       428
2       2       515
3       2       761
4       2       831
---------------------------------
   id               name                                         url  \
0   1       MALICE MIZER       http://www.last.fm/music/MALICE+MIZ

In [3]:
t = pd.read_table(user_taggedartists)
t[t.artistID == 995]

,userID,artistID,tagID,day,month,year
27,2,995,16,1,5,2009
28,2,995,17,1,5,2009
29,2,995,24,1,5,2009
30,2,995,25,1,5,2009
31,2,995,42,1,5,2009
32,2,995,43,1,5,2009
8939,73,995,956,1,8,2008
8940,73,995,960,1,9,2008
8941,73,995,961,1,9,2008
8942,73,995,962,1,8,2008


In [4]:
#Setting a total table of merge 2 sub tables
total_table = pd.read_table(user_taggedartists)
user_artists_df = pd.read_table(user_artists)
tags_df = pd.read_table(tags,encoding = "utf-8")
user_friends_df = pd.read_table(user_friends)

total_table.drop(["day","month","year"],axis = 1,inplace = True)
total_table = pd.merge(total_table,tags_df,on = "tagID")

def fun(word):
    word = "%s" % ' '.join(word)
    return word

artist_tags = total_table.groupby("artistID")["tagValue"].apply(fun)

list_word = []
for i in artist_tags[0:]:
    list_word.append(list(set(str.split(i))))
    
new_artist_tags = list(map(" ".join,list_word))
artist_tags_df = pd.DataFrame(artist_tags)
artist_tags_df["tagValue"] = new_artist_tags

In [5]:
total_table[total_table.artistID == 995]

,userID,artistID,tagID,tagValue
11701,1038,995,39,dance
12966,2,995,24,pop
13652,224,995,24,pop
18383,2,995,16,new wave
18525,117,995,16,new wave
19061,699,995,16,new wave
19901,1929,995,16,new wave
19978,2,995,17,synth pop
20305,2,995,25,80s
20525,117,995,25,80s


In [6]:
#artist_tags_df.head()
artist_tags_df['artistID'] = artist_tags_df.index
total_table = total_table.drop("tagValue",axis = 1)
total_table = pd.merge(artist_tags_df,total_table,how = "left")
total_table = total_table[["userID","artistID","tagID","tagValue"]]

In [7]:
#total_table = pd.merge(total_table,user_artists_df,how="left") #如果需要weight的数据，再去并
#total_table = pd.merge(total_table,user_friends_df,how="left") #如果需要friend的数据，再去并

In [8]:
total_table.head() ##最终总表

,userID,artistID,tagID,tagValue
0,681,1,139,lady japanese than jrock gaga better weeabo visual j-rock gothic kei
1,1545,1,139,lady japanese than jrock gaga better weeabo visual j-rock gothic kei
2,1730,1,139,lady japanese than jrock gaga better weeabo visual j-rock gothic kei
3,1929,1,139,lady japanese than jrock gaga better weeabo visual j-rock gothic kei
4,1984,1,139,lady japanese than jrock gaga better weeabo visual j-rock gothic kei


In [9]:
'''
#total_table.tagID
list_str = list(map(str,total_table.tagID))
tagID_df = pd.DataFrame(total_table.tagID)
tagID_df["str"] = list_str
'''

'\n#total_table.tagID\nlist_str = list(map(str,total_table.tagID))\ntagID_df = pd.DataFrame(total_table.tagID)\ntagID_df["str"] = list_str\n'

In [10]:
total_table_c = total_table.drop_duplicates(subset = ["userID","artistID","tagValue"])

In [11]:
if __name__ == '__main__':
    total_table_c.drop("tagID",axis = 1,inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
total_table_c.head()

,userID,artistID,tagValue
0,681,1,lady japanese than jrock gaga better weeabo visual j-rock gothic kei
1,1545,1,lady japanese than jrock gaga better weeabo visual j-rock gothic kei
2,1730,1,lady japanese than jrock gaga better weeabo visual j-rock gothic kei
3,1929,1,lady japanese than jrock gaga better weeabo visual j-rock gothic kei
4,1984,1,lady japanese than jrock gaga better weeabo visual j-rock gothic kei


In [13]:
total_table = pd.merge(total_table_c,user_artists_df,how="left") #如果需要weight的数据，再去并
total_table = pd.merge(total_table_c,user_friends_df,how="left") #如果需要friend的数据，再去并

In [14]:
print(total_table_c[total_table_c.artistID == 2].tagValue)

15    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
16    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
17    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
20    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
21    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
23    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
24    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
26    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
29    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
36    darkwave german electronic dark industrial ambient live emo vocal seen rock goth gothic true
37    dark

##### 方法1: 相似度评价 人和人之间的余弦值

In [15]:
total_table_c.head()
total_table_c[total_table_c.userID == 2]
all_tags = total_table_c.groupby("userID").tagValue.sum()

In [16]:
total_table_d = pd.DataFrame(all_tags)
total_table_d["userID"] = total_table_d.index

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

tf = TfidfVectorizer(ngram_range = (1,3),stop_words = 'english')
dtm = tf.fit_transform(total_table_d.tagValue)

In [19]:
from scipy.sparse.linalg import svds
u1,s1,vt1 = svds(dtm , k = 100)
u1.shape, s1.shape, vt1.shape

((1892, 100), (100,), (100, 328797))

In [20]:
pred = cosine_similarity(u1)
print(pred[0,].argsort())
total_table_d = total_table_d.reset_index(drop=True)

[1171 1098 1673 ..., 1791 1252    0]


In [21]:
total_table_d

tagValue  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [30]:
a = str(total_table_d[total_table_d.index == 0]['tagValue'])
b = str(total_table_d[total_table_d.index == 1791]['tagValue'])

In [31]:
a_l = str.split(a)
b_l = str.split(b)

In [32]:
len(list(set(a_l).intersection(set(b_l))))

106

##### 因为标签太杂，所以这种办法，被评价为太随意
##### 但我也没有分数，没法加权去用sim矩阵乘以评分向量，得到一个加权的东西

##### 仅仅根据用户和艺术家的对应关系，来评价
##### 先取后值1

In [34]:
total_table_c["like"] = 1.00

In [35]:
total_table_c
###sim = linear_kernel(dtm)
from scipy.sparse import csc_matrix
m = csc_matrix((total_table_c.like,(total_table_c.userID,total_table_c.artistID)))
m

<2101x18745 sparse matrix of type '<class 'numpy.float64'>'
	with 71064 stored elements in Compressed Sparse Column format>

In [36]:
from scipy.sparse.linalg import svds
u,s,vt = svds(m , k = 20)
u.shape, s.shape, vt.shape

((2101, 20), (20,), (20, 18745))

In [37]:
highest = np.argsort(vt[0,:])[-5:]
total_table_c[total_table_c.artistID.isin(highest)]
pred = np.dot(np.dot(u, np .diag(s)), vt)

In [49]:
pred

array([[  0.00000000e+00,   6.41681144e-18,   8.16315343e-18, ...,
          0.00000000e+00,   0.00000000e+00,   8.54680887e-19],
       [  0.00000000e+00,  -1.99537842e-18,  -2.39832560e-18, ...,
          0.00000000e+00,   0.00000000e+00,  -2.59953718e-19],
       [  0.00000000e+00,   6.47048279e-04,   3.38148559e-03, ...,
          0.00000000e+00,   0.00000000e+00,   5.47697979e-04],
       ..., 
       [  0.00000000e+00,   1.50015200e-17,   1.75742276e-17, ...,
          0.00000000e+00,   0.00000000e+00,   1.86774549e-18],
       [  0.00000000e+00,  -2.92290623e-03,   1.03445339e-02, ...,
          0.00000000e+00,   0.00000000e+00,   1.27409894e-03],
       [  0.00000000e+00,   2.39875734e-05,  -7.43665181e-04, ...,
          0.00000000e+00,   0.00000000e+00,   1.29555951e-04]])

In [50]:
recommended = np.argsort(pred[461, :])[-5:]
have_pay_attention = np.array(total_table_c[total_table_c.userID == 461].artistID)
list(set(recommended).difference(set(have_pay_attention)))

[65, 154, 1412]

In [53]:
print(have_pay_attention)
print(recommended)

[ 227  511  734 3081]
[  65 1412  511  154  227]


In [47]:
total_table_c[total_table_c.userID == 461]

,userID,artistID,tagValue,like
17130,461,227,wanna folk-rock pop-rock song 10 lovely piano jesus cute uk relax of track time 80s nineties excellent happy favourite they all gaga hey n your and cool that 1990's good mood 90's what world times to with male psicodelia life this room gods around classic alternative english lady hard brit nao artists kickass new in songs have jukebox amazing legends approved oldies favorite progressive popular crying g-e-n-i-o-s relaxing row wave the random politeconomist books harmonies absolutely punk a better top on can butterflies two will mellow reason phoda sweet u2 optimistic psychedelic sad rule vocalists more dance british mind-bursting is lyrics ever own feel sixties nostalgic genius listened amole perfect 1990s minutes day favourites blackbird see part guys 90s four full fab project already dad sick experimental 1960s chill wtf jazz made sentence ballad britpop cover just awesome-ness weekly 60s folk me youre under goku am epic my fun <3 first fave electronic mas i em mom addictive soundtrack for hit grandes make listen singalong grunge awesome these by harmonica ambient michelle post-punk brilliance arkadi 1960's novelty pop brilliant atmospheric comes masterpiece 70s soft you indian here catchy 1970 dreamy dazs lastfm memories 1008 best rock than invasion want at addicting roll chillout que instrumental love beatles story singer-songwriter scrubs music favorites lag title 40 definite fucking covers makes fav band sun albums acoustic hypnotic britrock joy realife beautiful taste old indie tracks morning so blues funny romantic 60's,1.0
38994,461,511,essential song piano oui champagne of 2000s 80s lirycs live forever all and cool that irish fail to male general sing seen 00s classic alternative english hard artists edit kickass new in songs x jukebox legends pedofinder it favorite wave the punk faves on touching genial u2 interesting factor vocalists beowulfs british awsome ever lyrics zone reminds again genius choice bands listened perfect sang eargasmic favourites eire 90s b-side project demo summer experimental chill ballad britpop cover slowies 1992 me epic my first electronic i diffrent diana soundtrack heat make listen heart grunge awesome repeat post-punk pop brilliant smile atmospheric masterpiece soft you single midnight days along over best singer uplifting dub at bono want aana roll instrumental fm love music singer-songwriter favorites leftovergrooves 500 zajebiste autumn band childhood acoustic beautiful old indie morning rock remix good,1.0
50271,461,734,summer cops the 1987 oscar ballad american slowies of 80s best christmas 1992 folk dub all sad epic singer-songwriter 1980 movie legend blues super awsome soundtrack jersey rain genius winning guitar acoustic 00s classic classical hard ambient 90s new songs legends 1984 rock remix oldies 1995 70s,1.0
106203,461,3081,female soul catchy song lovely piano emotion cute along memories over cheap happy folk sweet want at optimistic mine words chillout and love vocalists my singer-songwriter that first i to again addictive 4m4zinq pretty for sing nostalgic make listen f4nt4stic listened are acoustic 00s joy describe beautiful indie crazy songs amazing pop smile rides it you,1.0


In [57]:
total_table_c[total_table_c.artistID == 65].tagValue

3277    inlove hill song 10 lovely piano oui chlopcow cute uk tinosoft relax whitenoize of time muzyka happy favourite ben live special six cool and repellent good times fail musikfuerrundumdieuhr to never viaje male 2005 general seen sunshine 00s cold classic alternative dla egresados brit artists mp3 chillin naked new in songs x jukebox amazing kingdom pedofinder european oldies favorite it progressive row relaxing lpa wave tbu artist the state johnny cash elitist absolutely a question download top on britsong lll mellow god 2008 sweet one ohmydamn sad factor vocalists oh more british play ever own feel genius vacation bands listened perfect eno sang favourites private 90s mietkich xmas b-side project garden brian demo mix cities experimental 2002 chill sick beatiful isa ballad britpop cover just melancholic weekly gold folk religion caaamiiiliiitaaa under get overrated epic stuff fun my first galactic electronic i diana wistful soundtrack rain listen singalong tear-jearker awesome e